In [1]:
from bs4 import BeautifulSoup
html_doc = """
<html><head><title>Hello World</title></head>
<body><h2>Test Header</h2>
<p>This is a test.</p>
<a id="link1" href="/my_link1">Link 1</a>
<a id="link2" href="/my_link2">Link 2</a>
<p>Hello, <b class="boldtext">Bold Text</b></p>
</body></html>
"""
bs = BeautifulSoup(html_doc,features = "html.parser")
bs


<html><head><title>Hello World</title></head>
<body><h2>Test Header</h2>
<p>This is a test.</p>
<a href="/my_link1" id="link1">Link 1</a>
<a href="/my_link2" id="link2">Link 2</a>
<p>Hello, <b class="boldtext">Bold Text</b></p>
</body></html>

In [7]:
# 輸出排版後的 HTML 程式碼
print(bs.prettify())

<html>
 <head>
  <title>
   Hello World
  </title>
 </head>
 <body>
  <h2>
   Test Header
  </h2>
  <p>
   This is a test.
  </p>
  <a href="/my_link1" id="link1">
   Link 1
  </a>
  <a href="/my_link2" id="link2">
   Link 2
  </a>
  <p>
   Hello,
   <b class="boldtext">
    Bold Text
   </b>
  </p>
 </body>
</html>



In [3]:
# 網頁標題 HTML 標籤
bs.title

<title>Hello World</title>

In [61]:
# 網頁的標題文字
print(bs.title.string)#text也可以

Hello World


In [5]:
#我們可以使用 find_all 找出所有特定的 HTML 標籤節點
#再以 Python 的迴圈來依序輸出每個超連結的文字

a_tags = bs.find_all("a")
print(a_tags)
for tag in a_tags:
    print(tag.text)#string和text的結果一樣

[<a href="/my_link1" id="link1">Link 1</a>, <a href="/my_link2" id="link2">Link 2</a>]
Link 1
Link 2


In [24]:
#若要取出 HTML 節點的各種屬性
#可以使用 get，例如輸出每個超連結的網址（href 屬性）

for tag in a_tags:
    print(tag.get("href"))#也就是["href"]

/my_link1
/my_link2


In [25]:
#若要同時搜尋多種 HTML 標籤，可以使用 list 來指定所有的要列出的 HTML 標籤名稱

tags = bs.find_all(["a","b"])
tags

[<a href="/my_link1" id="link1">Link 1</a>,
 <a href="/my_link2" id="link2">Link 2</a>,
 <b class="boldtext">Bold Text</b>]

In [27]:
#limit 參數指定搜尋節點數量的上限值，這樣它就只會找出前幾個符合條件的節點
tags = bs.find_all(["a","b"],limit = 2)
tags

[<a href="/my_link1" id="link1">Link 1</a>,
 <a href="/my_link2" id="link2">Link 2</a>]

In [31]:
#如果只需要抓出第一個符合條件的節點，可以直接使用 find
a_tag = bs.find("a")
a_tag

<a href="/my_link1" id="link1">Link 1</a>

In [38]:
#預設的狀況下，find_all 會以遞迴的方式尋找所有的子節點
bs.find_all("title")

[<title>Hello World</title>]

In [39]:
#如果想要限制 find_all 只找尋次一層的子節點
#可以加上 recursive=False 關閉遞迴搜尋功能
bs.find_all("title",recursive = False)

[]

In [7]:
#根據網頁 HTML 元素的屬性來萃取指定的 HTML 節點
#例如搜尋 id 屬性為 link2 的節點

link2_tag = bs.find(id = "link2")
link2_tag

<a href="/my_link2" id="link2">Link 2</a>

In [44]:
#結合 HTML 節點的名稱與屬性進行更精確的搜尋
#例如搜尋 href 屬性為 /my_link1 的 a 節點
a_tag = bs.find_all("a",href = "/my_link1")
a_tag

[<a href="/my_link1" id="link1">Link 1</a>]

In [9]:
#搜尋屬性時，也可以使用正規表示法
import re
links = bs.find_all(href=re.compile("^/my_link\d"))
links

[<a href="/my_link1" id="link1">Link 1</a>,
 <a href="/my_link2" id="link2">Link 2</a>]

In [59]:
link = bs.find_all(href = re.compile(r"^/my_link\d"),
                   id = re.compile(r"link1"))
link

[<a href="/my_link1" id="link1">Link 1</a>]

In [62]:
#在 HTML5 中有一些屬性名稱若直接寫在 Python 的參數中會有一些問題
#例如 data-* 這類的屬性直接寫的話，就會產生錯誤訊息

data_soup = BeautifulSoup('<div data-foo="value">foo!</div>', 'html.parser')

# 錯誤的用法
data_soup.find_all(data-foo="value")

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (Temp/ipykernel_20560/1247491903.py, line 7)

In [64]:
#把屬性的名稱與值放進一個 dictionary 中
#再將此 dictionary 指定給 attrs 參數即可
data_soup = BeautifulSoup('<div data-foo="value">foo!</div>', 'html.parser')
data_soup.find_all(attrs = {"data-foo":"value"})

[<div data-foo="value">foo!</div>]

In [65]:
#class 是 Python 程式語言的保留字
#所以 Beautiful Soup 改以 class_ 這個名稱代表 HTML 節點的 class 屬性
b_tag = bs.find_all("b",class_ = "boldtext")
b_tag

[<b class="boldtext">Bold Text</b>]

In [67]:
#CSS 的 class 屬性也可以使用正規表示法搜尋
b_tag = bs.find_all("b",class_ = re.compile(r"^bold"))
b_tag

[<b class="boldtext">Bold Text</b>]

In [14]:
#一個 HTML 標籤元素可以同時有多個 CSS 的 class 屬性值
#而我們在以 class_ 比對時，只要其中一個 class 符合就算比對成功
css_soup = BeautifulSoup('<p class="body strikeout"></p>', 'html.parser')
p_tag = css_soup.find_all("p",class_ = re.compile(r"strikeout"))
print(p_tag)
p_tag = css_soup.find_all("p",class_ = re.compile(r"body strikeout"))
p_tag

[<p class="body strikeout"></p>]


[<p class="body strikeout"></p>]

In [18]:
#如果多個 class 名稱排列順序不同時，就會失敗
p_tag = css_soup.find_all("p",class_ = "strikeout body")
p_tag

[]

In [24]:
#遇到多個 CSS class 的狀況，建議改用 CSS 選擇器來篩選
p_tag = css_soup.select("p.strikeout.body")#有沒有p、strikeout都可以
#但一定要有.
print(p_tag)
item = p_tag[0]
item["class"]#.get("class")也可以

[<p class="body strikeout"></p>]


['body', 'strikeout']

In [28]:
#依據文字內容來搜尋特定的節點，可以使用 find_all 配合 string 參數
links_html = """
<a id="link1" href="/my_link1">Link One</a>
<a id="link2" href="/my_link2">Link Two</a>
<a id="link3" href="/my_link3">Link Three</a>
"""
bs = BeautifulSoup(links_html,"html.parser")
bs.find_all("a",string = "Link One")

[<a href="/my_link1" id="link1">Link One</a>]

In [30]:
#亦可使用正規表示法批配文字內容
bs.find_all("a",string = re.compile("^Link"))

[<a href="/my_link1" id="link1">Link One</a>,
 <a href="/my_link2" id="link2">Link Two</a>,
 <a href="/my_link3" id="link3">Link Three</a>]

In [5]:
# find_all 都是向下搜尋子節點，如果需要向上搜尋父節點的話
#可以改用 find_parents 函數（或是 find_parent）
from bs4 import BeautifulSoup as BS
html_doc = """
<body><p class="my_par">
<a id="link1" href="/my_link1">Link 1</a>
<a id="link2" href="/my_link2">Link 2</a>
<a id="link3" href="/my_link3">Link 3</a>
<a id="link3" href="/my_link4">Link 4</a>
</p></body>
"""
bs = BS(html_doc,"html.parser")#預設為html.parser?
link2_tag = bs.find(id = "link2")
link2_tag.find_parents("p")

[<p class="my_par">
 <a href="/my_link1" id="link1">Link 1</a>
 <a href="/my_link2" id="link2">Link 2</a>
 <a href="/my_link3" id="link3">Link 3</a>
 <a href="/my_link4" id="link3">Link 4</a>
 </p>]

In [8]:
#在同一層往前尋找特定節點，
#則可用 find_previous_siblings 函數（或是 find_previous_sibling）
link_tag = link2_tag.find_previous_siblings("a")#有s會有list
link_tag

[<a href="/my_link1" id="link1">Link 1</a>]

In [18]:
link_tag = link2_tag.find_previous()
link_tag

<a href="/my_link1" id="link1">Link 1</a>

In [11]:
#想要在在同一層往後尋找特定節點
#則可用 find_next_siblings 函數（或是 find_next_sibling），前者可以超過一個，後者只會有一個
link_tag = link2_tag.find_next_siblings("a")
link_tag

[<a href="/my_link3" id="link3">Link 3</a>,
 <a href="/my_link4" id="link3">Link 4</a>]

In [15]:
link_tag = link2_tag.find_all_next()
link_tag

[<a href="/my_link3" id="link3">Link 3</a>,
 <a href="/my_link4" id="link3">Link 4</a>]

In [ ]:
# Beautiful Soup 解析已經下載的 HTML 檔案
#可以直接將開啟的檔案交給 BeautifulSoup 處理
#Beautiful Soup 本身只是一個 HTML 解析工具，它並不負責下載網頁
#所以通常我們在開發爬蟲程式時，會搭配 requests 模組一同使用

In [5]:
import requests
from bs4 import BeautifulSoup
# response = requests.get("https://tw.yahoo.com/")
# 確認是否下載成功
if response.status_code == requests.codes.ok:
    bs = BeautifulSoup(response.text,"html.parser")
    #print(bs)
    #print(bs.prettify())
    stories = bs.find_all("a",class_ = "active_V(v)")
    #print(stories)
    for s in stories:
        print("標題:" + s.text)
        print("網址:" + s.get("href"))


標題:俄軍實際陣亡多少？報告揭曉
網址:https://tw.news.yahoo.com/%E4%BF%84%E8%BB%8D%E5%AF%A6%E9%9A%9B%E9%99%A3%E4%BA%A1%E5%A4%9A%E5%B0%91%E4%BA%BA%EF%BC%9F%E4%BF%84%E7%8D%A8%E7%AB%8B%E5%AA%92%E9%AB%94%E6%9F%A5%E5%87%BA%E8%BF%915%E8%90%AC%E4%BA%BA-134038599.html
標題:刀口下救美 父子3人身分曝光
網址:https://tw.news.yahoo.com/%E7%BE%A9%E5%8B%87%E7%88%B6%E5%AD%90%E6%95%91%E7%BE%8E%E7%AB%9F%E6%98%AF%E9%80%99%E8%BA%AB%E5%88%86-%E5%8F%B0%E6%9D%B1%E8%AA%BF%E6%9F%A5%E7%AB%99%E5%8F%B8%E6%A9%9F%E8%82%89%E8%BA%AB%E6%93%8B%E6%85%98%E6%A1%88-045718401.html
標題:關鍵落差⋯戳「民進黨警訊」
網址:https://tw.news.yahoo.com/%E8%B3%B4%E6%B8%85%E5%BE%B7-%E6%B0%91%E9%80%B2%E9%BB%A8%E6%94%AF%E6%8C%81%E5%BA%A6-%E6%B8%B8%E7%9B%88%E9%9A%86-%E6%B0%91%E9%80%B2%E9%BB%A8-%E8%AD%A6%E8%A8%8A-064036046.html
標題:陳凱倫涉性侵 中職永不錄用
網址:https://tw.sports.yahoo.com/news/%E9%99%B3%E5%87%B1%E5%80%AB%E5%B0%8D%E9%9A%8A%E5%8F%8B%E8%80%81%E5%A9%86%E4%BC%B8%E7%8B%BC%E7%88%AA%E8%A2%AB%E5%88%A4%E5%88%91-%E5%89%8D%E6%9D%B1%E5%AE%B6%E5%AF%8C%E9%82%A6%E6%82%8D%E5%B0%87%E7%99%BC%E8%81%B2%E6%98

In [ ]:
# Google 的網頁會因為瀏覽器（User-Agent）不同而產生不同的結果
#所以在觀察程式碼的時候
#最好是使用 Beautiful Soup 的 prettify 
#把抓回來的 HTML 原始碼排版後印出來，這樣看會比較準確。

In [1]:
import requests
from bs4 import BeautifulSoup

google_url = "https://www.google.com.tw/search"
#my_params = {"q":"寒流"}

# response = requests.get(google_url,params = {"q":"北科大"})

#print(response.text)


if response.status_code == requests.codes.ok:
    bs = BeautifulSoup(response.text,features = "html.parser")
    #print(bs.prettify())
#     items = bs.find_all("a")#,class_ = "BVG0Nb")#,attrs = {"class" : "WlydOe"})
    #print(items)
#     for item in items:
        #print(item.prettify())
#         title = item
        #print(item.text)
    
    
    
    
#     items = bs.find_all("a",class_ = "DKV0Md")
#     print(items)
#     for i in items:
#         print("標題:",i.text)
#         print("網址:",i.get("href"))
        #<h3 class="LC20lb MBeuO DKV0Md">寒流</h3>
  



In [2]:
results = bs.find_all("a",class_ = "BVG0Nb")
for result in results:
    title = result.find("span",class_ = "rQMQod")
    try:
        print("標題:" + title.text)
        print("連結:" + result.get("href"))
        print()
    except AttributeError:
        print("沒有這個型別")

標題:影／北科大白飯之亂！資財營再列7點強調「用餐體驗差」：絕不是飯桶系
連結:/url?q=https://tw.news.yahoo.com/%25E5%25BD%25B1-%25E5%258C%2597%25E7%25A7%2591%25E5%25A4%25A7%25E7%2599%25BD%25E9%25A3%25AF%25E4%25B9%258B%25E4%25BA%2582-%25E8%25B3%2587%25E8%25B2%25A1%25E7%2587%259F%25E5%2586%258D%25E5%2588%25977%25E9%25BB%259E%25E5%25BC%25B7%25E8%25AA%25BF-%25E7%2594%25A8%25E9%25A4%2590%25E9%25AB%2594%25E9%25A9%2597%25E5%25B7%25AE-%25E7%25B5%2595%25E4%25B8%258D%25E6%2598%25AF%25E9%25A3%25AF%25E6%25A1%25B6%25E7%25B3%25BB-225012825.html&sa=U&ved=2ahUKEwiKrI751IaAAxUMfN4KHZnfAN0Q0PADegQIBxAD&usg=AOvVaw1-u2MsgRrfuRVipyaWxyuq

標題:熱炒店「白飯之亂」學生不想忍！北科大校方首發聲明：協助學生面對消費爭議
連結:/url?q=https://www.storm.mg/lifestyle/4828418&sa=U&ved=2ahUKEwiKrI751IaAAxUMfN4KHZnfAN0Q0PADegQIBxAH&usg=AOvVaw1RMxViiUUiK6WbGrq7U6Lc

標題:北科大別衝動！研究生「30秒吞2.5顆饅頭」 大胃王比賽奪第一後暴斃
連結:/url?q=https://www.setn.com/News.aspx%3FNewsID%3D1321449&sa=U&ved=2ahUKEwiKrI751IaAAxUMfN4KHZnfAN0Q0PADegQIBxAL&usg=AOvVaw14xV2GBz6pxv_LdW_ngk0V

標題:北科大別衝動！研究生「30秒吞2.5顆饅頭」 大胃王比賽奪第一後暴斃
連結:/url?q=

In [26]:
links = bs.find_all("a",attrs = {"data-ved":"2ahUKEwi4ppqqpYSAAxUEAYgKHdVKDvcQFnoECAcQAg"})
print(links[0].text)

國立臺北科技大學www.ntut.edu.tw


In [20]:
elements = bs.select("a.Q71vJc")
for element in elements:
    print(element.text)

北科大校園入口
北科大研究所
北科大排名
北科大行事曆
北科大分數
北科大校務系統
北科大碩士
北科大地址
